In [1]:
import spacy
import csv
from collections import namedtuple
from spacy.matcher import PhraseMatcher

In [2]:
o = ""

for page in range(1,60):
    with open('edited_from_ocr/Complaint-p{}-normal.txt'.format(str(page)),'r') as output:
        o = "\n".join([o, output.read()])

In [3]:
o = o.replace("\n", " ").replace("Case 1:18-cv-03501 Document 1 Filed 04/20/18 ", "")\
    .replace("$$", "§§").replace(" $ ", " § ").replace(".$ ", ". § ").replace("С", "C")\
    .replace(" S ", " § ").replace(" SS", " §§").replace(" 55 "," §§ ").replace("US.C.", "U.S.C.")\
    .replace("U.S.C. 8 ", "U.S.C. § ").replace("&well","and well").replace(" & ",", ")

In [4]:
for n in range(66):
    o = o.replace(f"Page {n} of 66", f'**{n}**')
    # o = o.replace(f"Page {n} of 66", f'<page number="{n}" />')

In [5]:
filename = "./edited_from_ocr/full_complaint.txt"

with open(filename, 'w') as fout:
    fout.write(o)

In [6]:
# Spacy parsing probably needs to be done before tags are added.

o = "<complaint>" + o + "</complaint>"

In [7]:
def tag_allegations(s: str,first: int,last: int):
    s = s.replace(f" {first}. ", f"""<allegation para="{first}">""", 1)
    for n in range(first+1, last+1):
        s = s.replace(f" {n}. ", f"""</allegation><allegation para="{n}">""", 1)
    s = s.replace(f" {last+1}. ", f"""</allegation> {last+1}. """, 1) # not good, consumes the next heading.
    return s

o = tag_allegations(o, 146, 153)

In [8]:
# o

In [9]:
parties = []

with open("./data/parties.csv", newline="") as infile:
    reader = csv.reader(infile)
    Party = namedtuple("Party", next(reader))  # get names from column headers
    for data in map(Party._make, reader):
        parties.append(data)
    

In [10]:
parties

[Party(role='Plaintiff', name='Democratic National Committee', short_name='DNC'),
 Party(role='Defendant', name='Russian Federation', short_name='Russia'),
 Party(role='Defendant', name='General Staff of the Armed Forces of the Russian Federation', short_name='GRU'),
 Party(role='Defendant', name='GRU operative using the pseudonym "Guccifer 2.0"', short_name='GRU Operative #1'),
 Party(role='Defendant', name='Aras Iskenerovich Agalarov', short_name='Aras Agalarov'),
 Party(role='Defendant', name='Emin Araz Agalarov', short_name='Emin Agalarov'),
 Party(role='Defendant', name='Joseph Mifsud', short_name='Mifsud'),
 Party(role='Defendant', name='WikiLeaks', short_name='Wiki Leaks'),
 Party(role='Defendant', name='Julian Assange', short_name='Assange'),
 Party(role='Defendant', name='Donald J. Trump for President, Inc.', short_name='Trump Campaign'),
 Party(role='Defendant', name='Donald J. Trump, Jr.', short_name='Trump, Jr.'),
 Party(role='Defendant', name='Paul J. Manafort, Jr.', short

In [11]:
nlp = spacy.load('en_core_web_sm')
doc = nlp(o)

In [12]:
# This is probably overkill. I should just use string methods to highlight/tag the names.

party_matches = []

def on_match(matcher, doc, uid, matches):
    print(f'{uid} matched!')
    # party_matches.append((uid, matches, "Party"))

matcher = PhraseMatcher(nlp.vocab)
for p in parties:
    if len(p.name) < 10:
        matcher.add(p.short_name, on_match, nlp(p.short_name), nlp(p.name))
    else: matcher.add(p.short_name, on_match, nlp(p.short_name))
    
# matches = matcher(doc)

In [13]:
Entity = namedtuple("Entity", "text start_char end_char label_")

In [14]:
entities = [Entity(e.text, e.start_char, e.end_char, e.label_) for e in doc.ents
            if e.label_ not in ("CARDINAL", "ORDINAL", "DATE", "MONEY", "TIME", "LAW", "FAC")]

In [15]:
# entities

In [16]:
for sent in list(doc.sents)[460:470]:
    print(sent.text)

CrowdStrike performed forensic analysis on the DNC's computer network and servers.
CrowdStrike also set up a system for monitoring the ongoing attack on Plaintiffs computer system and to alert the DNC to future attacks.
85.
As a result of the persistence of the Russian state-sponsored infiltration, in order to remove the unauthorized users from its network, the DNC was required to decommission more than 140 servers, remove and reinstall all software, including the operating systems, for more than 180 computers, and rebuild at least 11 servers.  
*
*27
*
* G. Forensic Evidence Confirms Russia's Attack on the DNC's Network 86.
Both CrowdStrike's forensic analysis and the U.S. Government concluded that the DNC's computer systems had been hacked by two independent, sophisticated Russian state sponsored adversaries, both with a nexus to Russia's intelligence services.5
The forensic analysts tracked the hacking activities of these adversaries by assigning them code names: "Cozy Bear" and "Fa

In [17]:
doc.ents[1]

THE UNITED STATES DISTRICT COURT

In [18]:
type(doc.ents[1])

spacy.tokens.span.Span

In [19]:
import xml.etree.ElementTree as ET
root = ET.fromstring(o)

In [20]:
root.getchildren()

[<Element 'allegation' at 0x10ae96728>,
 <Element 'allegation' at 0x10ae96cc8>,
 <Element 'allegation' at 0x10ae96688>,
 <Element 'allegation' at 0x10ae967c8>,
 <Element 'allegation' at 0x10ae96638>,
 <Element 'allegation' at 0x10ae96278>,
 <Element 'allegation' at 0x10ae96e58>]

In [21]:
for a in root.getchildren():
    print(a.text)
    print("")

Plaintiff realleges and incorporates by reference all prior paragraphs of this Complaint and paragraphs in the counts below as though set forth fully herein.

The DNC's computers are involved in interstate and foreign commerce and communication, and are protected computers under 18 U.S.C. 1030(e)(2).

On information and belief, Russia, the GRU, and GRU Operative #1 knowingly and intentionally accessed the DNC's computers without authorization or in excess of authorization, and thereby obtained and used valuable information from those computers irn violation of 18 U.S.C. § 1030(a)(2)(C). Such information included, but was not limited to: private, politically sensitive communications between the DNC and Democratic stakeholders and candidates, confidential donor data; confidential campaign strategy plans; opposition and policy research; and documents regarding planned political events, including fundraisers and rallies. The information was used to advance the plan to denigrate the Democra